In [4]:
import pandas as pd
import numpy as np
from openai.embeddings_utils import get_embedding
from openai.embeddings_utils import cosine_similarity
import openai
import os

# Set your variables here
openai.api_key = 'enteryourkey'
file_name= "course.csv"

# Functions

In [5]:
embedding_model = 'text-embedding-ada-002'

def get_embedding(model: str, text: str) -> list[float]:
  """Get embeddings for given input string

  Args:
      model (str): Name of the model
      text (str): Text needs to be converred

  Returns:
      list[float]: List of array
  """
  result = openai.Embedding.create(
    model = model,
    input = text
  )
  return result['data'][0]['embedding']

def get_answer(my_input:str,df_local):
  """Compare arrays and return result based if similarity is above threshold

  Args:
      my_input (str): _description_
      df_local (_type_): _description_

  Returns:
      _type_: _description_
  """
  input_embedding_vector = get_embedding(embedding_model, my_input)
  
  df_local['embedding2'] = df_local['embedding'].apply(eval).apply(np.array)
  df_local['similarity'] = df_local['embedding2'].apply(lambda x: cosine_similarity(x, input_embedding_vector))
  # Find the highest similarity value in the dataframe column 'similarity'
  highest_similarity = df_local['similarity'].max()

  # If the highest similarity value is equal or higher than 0.9 then print the 'content' with the highest similarity
  if highest_similarity >= 0.8:
      fact_with_highest_similarity = df_local.loc[df_local['similarity'] == highest_similarity, 'completion']
      return str(fact_with_highest_similarity.values)
  else:
    return "Sorry please ask your question again."

# Convert to Embedding
- Check if file exists
- Read the file and save embedding as new file.
- Load file to df for comparison

In [7]:
embeddings_name = os.path.splitext(file_name)[0] + "_embeddings.csv"
if os.path.exists(embeddings_name):
  df = pd.read_csv(embeddings_name)
else:
  df = pd.read_csv(file_name)
  df.head(1)
  df['embedding'] = df['prompt'].apply(lambda x: get_embedding(embedding_model,x))
  df.to_csv(embeddings_name)
  df = pd.read_csv(embeddings_name)

In [13]:
my_input = 'job after training?'
get_answer(my_input,df)

"['Absolutely Yes! Our Selenium Automation Online training is designed to equip you with the skill set needed to secure a great job in the field. Our comprehensive course material, in-depth curriculum and hands-on exercises prepare you for challenges in the real-world. Our specialists will also guide you through the interview process. Finally, our certification ensures that you will become an employable asset in the industry.']"